In [1]:
!pip install transformers gradio accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import torch

In [3]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
chat_model_name = "tiiuae/falcon-7b-instruct"  # Or use 3B if low on RAM


In [7]:
tokenizer = AutoTokenizer.from_pretrained(chat_model_name)

# Define the desired torch_dtype and device before using them
torch_dtype = torch.float16
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model using device_map="auto" which handles device placement
model = AutoModelForCausalLM.from_pretrained(
    chat_model_name,
    torch_dtype=torch_dtype,
    device_map="auto" if device == "cuda" else None
)
# Removed the .to(device) call as device_map="auto" handles device placement

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def chat_with_model(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output = model.generate(input_ids, max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response[len(prompt):].strip()


In [9]:
def chat_with_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()

In [10]:
def generate_summary(entry):
    summary = summarizer(entry, max_length=60, min_length=20, do_sample=False)
    return summary[0]['summary_text']

In [11]:
def ai_therapist_response(entry):
    prompt = f"You are a kind and understanding therapist. The user said: '{entry}'. Respond compassionately and supportively."
    return chat_with_model(prompt)


In [15]:
def suggest_actions(entry):
    if not entry.strip():
        return "Please enter a valid journal entry."

    prompt = f"Read this journal entry: '{entry}'. Suggest a few healthy coping strategies or self-care tips to support mental well-being."

    try:
        return chat_with_model(prompt)
    except Exception as e:
        return f"⚠️ Error generating suggestions: {str(e)}"


In [19]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI Mental Health Journal (Free Version)")
    gr.Markdown("Write your thoughts. Get a summary, supportive reply, and self-care tips.")

    entry = gr.Textbox(lines=10, label="📝 Your Journal Entry")
    btn = gr.Button("🧠 Analyze Entry")

    summary_out = gr.Textbox(label="✍️ Summary")
    therapist_out = gr.Textbox(label="💬 Therapist's Response")
    suggestion_out = gr.Textbox(label="🌿 Wellness Suggestions")

    # ✅ These must be INSIDE the 'with gr.Blocks()' block
    btn.click(fn=generate_summary, inputs=entry, outputs=summary_out)
    btn.click(fn=ai_therapist_response, inputs=entry, outputs=therapist_out)
    btn.click(fn=suggest_actions, inputs=entry, outputs=suggestion_out)

# ✅ This can stay outside
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://670a7d9f94c3ddbe8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
